In [1]:
# Thsi is the code used for the paper results
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import os
import sys
import numpy as np 
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct
import matplotlib.pyplot as plt


In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 10,
    "exp_name": ct.generate_readable_short_id("Paper_v2_feature_size_TCE_KL"),
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 10,

    # "test_split": 0.1,
    # "calib_split": 0.11,
    
    "plot": False,
    "calib_methods": ["RF_d", "RF_opt", "RF_large",
                      "Platt", "ISO", "Beta", "CRF", "VA",
                      "tlr", "Rank", "Elkan",                      
                      ],
    
    "metrics": ["acc", "brier", "ece", "logloss", "tce"],

    #data
    "data_name": "synthetic",
    "plot_data": True,
    "data_size": 1000,
    "n_features": 2,

    "class1_mean_min":0, 
    "class1_mean_max":1,
    "class2_mean_min":0, 
    "class2_mean_max":1, 

    "class1_cov_min":1, 
    "class1_cov_max":2,
    "class2_cov_min":1, 
    "class2_cov_max":2, 


    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth": np.arange(2, 10).tolist(),
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf": np.arange(1, 11).tolist(),
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false

    "exp_key": "n_features",
    "exp_values":  np.arange(3, 50, 2).tolist()
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

Success in 3 n_features accuracy 0.745 runs 5179
exp_param 3 done
Success in 5 n_features accuracy 0.745 runs 180
exp_param 5 done
exp_param 7 done
Success in 9 n_features accuracy 0.745 runs 6
exp_param 9 done
Success in 11 n_features accuracy 0.745 runs 3
exp_param 11 done
Success in 13 n_features accuracy 0.745 runs 108
exp_param 13 done
Success in 15 n_features accuracy 0.745 runs 3
exp_param 15 done
Success in 17 n_features accuracy 0.745 runs 86
exp_param 17 done
Success in 19 n_features accuracy 0.745 runs 27
exp_param 19 done
Success in 21 n_features accuracy 0.745 runs 16
exp_param 21 done
Success in 23 n_features accuracy 0.745 runs 46
exp_param 23 done
Success in 25 n_features accuracy 0.745 runs 30
exp_param 25 done
Success in 27 n_features accuracy 0.745 runs 160
exp_param 27 done
exp_param 29 done
exp_param 31 done
Success in 33 n_features accuracy 0.745 runs 79
exp_param 33 done
Success in 35 n_features accuracy 0.745 runs 24
exp_param 35 done
exp_param 37 done
Success i

/Users/mhshaker/Documents/PhD/Projects/RF_ranking_calibration/RFcalib/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


exp_param 47 done
exp_param 49 done


In [4]:
plot_calib_methods = params["calib_methods"].copy()
# plot_calib_methods.remove("RF_large")
plot_calib_methods.remove("Elkan")
# plot_calib_methods.remove("RF")
plot_calib_methods

['RF_d',
 'RF_opt',
 'RF_large',
 'Platt',
 'ISO',
 'Beta',
 'CRF',
 'VA',
 'tlr',
 'Rank']

In [5]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=False, 
                                    std=True)

path = f"results/{params['exp_name']}/features"
if not os.path.exists(path):
    os.makedirs(path)

for metric in params["metrics"]: #
    ax = tables[metric][plot_calib_methods].plot()
    ax.set_xlabel(params["exp_key"])
    ax.set_ylabel(metric)
    plt.savefig(f"{path}/f_{metric}.pdf", format='pdf', transparent=True)
    plt.close() 
